In [ ]:
!pip3 install transformers sentencepiece pandas gensim

In [ ]:
import pandas as pd 
import numpy as np 
import torch
from transformers import AutoTokenizer, AutoModel
import gc
import gensim

In [ ]:
cols = ['item_id', 'title', 'description']
text_cols = ['title', 'description']
data_chunks = pd.read_csv('/storage/data/simple_features.csv', chunksize=100000)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny")
model.cuda()  # uncomment it if you have a GPU

def embed_bert_cls(text):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output[-2][:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

doc2vec_models = {col: gensim.models.Doc2Vec.load('/storage/data/doc2vec_{}'.format(col)) for col in text_cols}

for i, data in enumerate(data_chunks):
  data = data[cols].fillna('продать')
  for col in text_cols:
      texts = data[col].tolist()
      bert_embeds = [embed_bert_cls(str(text)) for text in texts]
      doc2vec_embeds = [doc2vec_models[col].infer_vector(gensim.utils.simple_preprocess(str(text))) for text in texts]
      data = pd.concat([data, 
                        pd.DataFrame(bert_embeds, columns=['{}_mbert_{}'.format(col, i) for i in range(len(bert_embeds[0]))]),
                        pd.DataFrame(doc2vec_embeds, columns=['{}_doc2vec_{}'.format(col, i) for i in range(len(doc2vec_embeds[0]))])], axis=1)
      texts, bert_embeds, doc2vec_embeds = None, None, None
      gc.collect()
  data = data.drop(text_cols, axis=1)
  data.to_csv('/storage/data/text_train.csv',
                  header = True if i == 0 else False, index=False,
                  mode = 'w' if i == 0 else 'a')
  data = None
  gc.collect()